In [89]:
from requests import get
from pprint import PrettyPrinter
import pyodbc
import pandas as pd
import sqlalchemy as db
import urllib
import datetime as dt

printer = PrettyPrinter()

In [ ]:
# NBA Stats - JSON Endpoints
# http://data.nba.net/10s/prod/v1/2021/players.json > Player data for a specific year
# https://data.nba.net/10s/prod/v1/2021/teams.json > Team data for a specific year
# http://data.nba.net/data/10s/prod/v1/20180928/0011800001_boxscore.json > Box score data for a specific game (needs date and game id)
# https://data.nba.net/10s/prod/v1/20180929/scoreboard.json > Box scores for all games on a specific day (needs date)
# https://data.nba.net/10s/prod/v1/2021/schedule.json > Schedule data for a specific year

In [ ]:
# GETS TEAM ROSTER DATA
# team_roster_endpoint = api.data_links['teamRoster']
# team_roster_endpoint = team_roster_endpoint.replace('{{teamUrlCode}}','raptors')

# test_data = get(base_url + team_roster_endpoint).json()
# print(len(test_data['league']['standard']['players']))

In [ ]:
# CONNECTING TO MS SQL SERVER DATABASE
# import urllib
# import sqlalchemy as db

# driver = "{ODBC Driver 18 for SQL Server}"
# server = "asqlsrv-nbadashboard-dev-canadacentral-001.database.windows.net"
# database = "asqldb-nbadashboard-dev-canadacentral-001"
# username = "db_admin"
# password = "Salazar5991!"

# connection_string = f"Driver={driver};Server=tcp:{server},1433;Database={database};Uid={username};Pwd={password};Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30";

# params = urllib.parse.quote_plus(connection_string)

# engine = db.create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)

In [ ]:
# GET BOX SCORE DATA
# api.data_links

# api.get_game_data('20211107', '0022100139').keys()
# test_data = get(base_url + '/prod/v1/20211107/0022100139_mini_boxscore.json').json()['basicGameData']
# api.schedule_data[204]

In [ ]:
# EXTRACTS NESTED DATA
for player_dict in player_list_of_dicts:

    # EXTRACTS DATA FROM "teamSitesOnly" DICT
    team_sites_only_dict = player_dict['teamSitesOnly']

    for key in team_sites_only_dict:
        player_dict[f"teamsitesonly_{key}"] = team_sites_only_dict[key]

    # EXTRACTS DATA FROM "draft" DICT
    draft_dict = player_dict['draft']

    for key in draft_dict:
        player_dict[f"draft_{key}"] = draft_dict[key]

    # EXTRACT DATA from "teams" LIST OF DICTS
    teams_dict = player_dict['teams']
    counter = 1
    
    for team in teams_dict:
        for key in team:
            player_dict[f"teamhist_{counter}_{key}"] = team[key]
        counter += 1

In [17]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
# player_data_df.loc[player_data_df['teamId'] == '1610612748']

In [96]:
class NBAStatsAPI:

    base_url = "https://data.nba.net"
    all_json = "/prod/v1/today.json"

    
    def __init__(self):
        # Dict with JSON endpoints
        self._data_links = self._get_all_data_links()
        # List of dicts with each dict containing data for a team
        self._team_data = self._get_team_data()
        # List of dicts with each dict containing data for a player
        self._player_data = self._get_player_data()
        # List of dicts with each dict containing data for a scheduled game
        self._schedule_data = self._get_schedule_data()
        # Date data was last refreshed
        self._date_last_refreshed = dt.datetime.strptime(self._data_links['currentDate'], '%Y%m%d')

    @property
    def data_links(self):
        return self._data_links
    
    @property
    def team_data(self):
        return self._team_data

    @property
    def player_data(self):
        return self._player_data

    @property
    def schedule_data(self):
        return self._schedule_data

    def _get_all_data_links(self):
        data = get(NBAStatsAPI.base_url + NBAStatsAPI.all_json).json()
        links = data['links']

        return links

    def _get_team_data(self):
        team_data_endpoint = get(NBAStatsAPI.base_url + self.data_links['teams']).json()
        team_data = team_data_endpoint['league']['standard']
        team_data = list(filter(lambda x: x['isNBAFranchise'] == True, team_data))

        return team_data

    def _get_player_data(self):
        player_data_endpoint = get(NBAStatsAPI.base_url + self.data_links['leagueRosterPlayers']).json()
        player_data = player_data_endpoint['league']['standard']

        # EXTRACTS NESTED PLAYER DATA
        for player_dict in player_data:

            # EXTRACTS DATA FROM "teamSitesOnly" DICT
            team_sites_only_dict = player_dict['teamSitesOnly']

            for key in team_sites_only_dict:
                player_dict[f"teamsitesonly_{key}"] = team_sites_only_dict[key]

            # EXTRACTS DATA FROM "draft" DICT
            draft_dict = player_dict['draft']

            for key in draft_dict:
                player_dict[f"draft_{key}"] = draft_dict[key]

            # EXTRACT DATA from "teams" LIST OF DICTS
            teams_dict = player_dict['teams']
            counter = 1
            
            for team in teams_dict:
                for key in team:
                    player_dict[f"teamhist_{counter}_{key}"] = team[key]
                counter += 1
        
        return player_data
    
    def _get_schedule_data(self):
        schedule_data_endpoint = get(NBAStatsAPI.base_url + self.data_links['leagueSchedule']).json()
        schedule_data = schedule_data_endpoint['league']['standard']

        return schedule_data

    def get_game_data(self, game_date: str, game_id: str):
        game_data_link = self.data_links['boxscore']
        game_data_link = game_data_link.replace('{{gameDate}}', f'{game_date}')
        game_data_link = game_data_link.replace('{{gameId}}', f'{game_id}')

        game_data_endpoint = get(NBAStatsAPI.base_url + game_data_link).json()
        game_data = game_data_endpoint['basicGameData']

        return game_data

    def __str__(self):
        return f"Data Last Refreshed On: {self._date_last_refreshed.strftime('%B %d, %Y')}"

In [6]:
# Instantiate API Object
api = NBAStatsAPI()


In [94]:
type(api.player_data)

list

In [98]:
base_url = "https://data.nba.net"
# all_json = "/prod/v1/today.json"
data = get(NBAStatsAPI.base_url + NBAStatsAPI.all_json).json()
data_links = data['links']

player_data_endpoint = get(base_url + data_links['leagueRosterPlayers']).json()
player_data = player_data_endpoint['league']['standard']
        

In [100]:
# Keys: teamSitesOnly, teams, draft
# player_data[0]['teamSitesOnly']

player_list_of_dicts = player_data[0:2]

In [105]:
player_list_of_dicts[0]

{'firstName': 'Precious',
 'lastName': 'Achiuwa',
 'temporaryDisplayName': 'Achiuwa, Precious',
 'personId': '1630173',
 'teamId': '1610612761',
 'jersey': '5',
 'isActive': True,
 'pos': 'F',
 'heightFeet': '6',
 'heightInches': '8',
 'heightMeters': '2.03',
 'weightPounds': '225',
 'weightKilograms': '102.1',
 'dateOfBirthUTC': '1999-09-19',
 'teamSitesOnly': {'playerCode': 'precious_achiuwa',
  'posFull': 'Forward',
  'displayAffiliation': 'Memphis/Nigeria',
  'freeAgentCode': ''},
 'teams': [{'teamId': '1610612748',
   'seasonStart': '2020',
   'seasonEnd': '2020'},
  {'teamId': '1610612761', 'seasonStart': '2021', 'seasonEnd': '2021'}],
 'draft': {'teamId': '1610612748',
  'pickNum': '20',
  'roundNum': '1',
  'seasonYear': '2020'},
 'nbaDebutYear': '2020',
 'yearsPro': '1',
 'collegeName': 'Memphis',
 'lastAffiliation': 'Memphis/Nigeria',
 'country': 'Nigeria',
 'teamsitesonly_playerCode': 'precious_achiuwa',
 'teamsitesonly_posFull': 'Forward',
 'teamsitesonly_displayAffiliation

In [108]:
player_list_of_dicts[0].get('teamSitesOnly')

{'playerCode': 'precious_achiuwa',
 'posFull': 'Forward',
 'displayAffiliation': 'Memphis/Nigeria',
 'freeAgentCode': ''}

In [109]:
test_dict = None

for key in test_dict:
    print(key)

TypeError: 'NoneType' object is not iterable

In [99]:
# EXTRACTS NESTED PLAYER DATA
for player_dict in player_data:

    # EXTRACTS DATA FROM "teamSitesOnly" DICT
    team_sites_only_dict = player_dict.get('teamSitesOnly')
    
    if team_sites_only_dict != None:
        for key in team_sites_only_dict:
            player_dict[f"teamsitesonly_{key}"] = team_sites_only_dict[key]

    # EXTRACTS DATA FROM "draft" DICT
    draft_dict = player_dict.get('draft')

    if draft_dict != None:
        for key in draft_dict:
            player_dict[f"draft_{key}"] = draft_dict[key]

    # EXTRACT DATA from "teams" LIST OF DICTS
    teams_dict = player_dict.get('teams')

    if teams_dict != None:
        counter = 1
    
        for team in teams_dict:
            for key in team:
                player_dict[f"teamhist_{counter}_{key}"] = team[key]
            counter += 1

KeyError: 'teamSitesOnly'